# Project 15: Text2SQL via Prompt Engineering

# Text-to-SQL: Bridging the Gap Between Human Language and Databases


Text-to-SQL, also known as Natural Language to SQL (NL2SQL), is a rapidly evolving technology that translates natural, everyday language into Structured Query Language (SQL) commands. This innovative approach empowers users to interact with and retrieve data from databases simply by asking questions in plain English, eliminating the need for specialized knowledge of complex SQL syntax.

At its core, Text-to-SQL acts as an intelligent translator. It leverages the power of artificial intelligence, particularly **Natural Language Processing (NLP)** and sophisticated **AI models**, to understand the user's intent and generate the corresponding SQL query. This process allows individuals without a technical background to explore and analyze data, thereby democratizing data access within an organization.

## How It Works: From a Simple Question to a Complex Query

The conversion of a user's question into an executable SQL query involves a multi-step process:

1.  **Natural Language Understanding (NLU):** The system first analyzes the user's input to decipher its meaning. This involves identifying key entities (like specific columns or tables), the relationships between them, and the user's ultimate goal (e.g., to filter, aggregate, or sort data).

2.  **Schema Linking:** Once the intent is understood, the system maps the identified entities from the natural language question to the specific tables and columns within the database's schema. This is a critical step to ensure the generated query is accurate and relevant to the available data structure.

3.  **SQL Generation:** With the user's intent and the relevant database schema components identified, the AI model constructs the appropriate SQL query. This can range from a simple `SELECT` statement to a complex query involving multiple `JOIN`s, `WHERE` clauses, and aggregate functions.

4.  **Query Execution and Response:** The generated SQL query is then executed against the database. The retrieved data is presented back to the user in a clear and understandable format, often as a table, chart, or a natural language summary.

# Tangent 1: You should know about Mockaroo

http://mockaroo.com

In [2]:
! curl "https://api.mockaroo.com/api/dde01370?count=1000&key=11149690" > "customers.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   97k    0   97k    0     0  51850      0 --:--:--  0:00:01 --:--:-- 51850


In [3]:
! curl "https://api.mockaroo.com/api/8ba6f630?count=1000&key=11149690" > "products.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  684k    0  684k    0     0   314k      0 --:--:--  0:00:02 --:--:--  314k


In [4]:
! curl "https://api.mockaroo.com/api/6fa67fe0?count=3000&key=11149690" > "orders.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  230k    0  230k    0     0  31276      0 --:--:--  0:00:07 --:--:-- 39024


## Setup database

In [5]:
import sqlite3
import pandas as pd
import os

In [6]:
# Define SQL schemas for creating tables
customers_schema = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    address VARCHAR(50),
    city VARCHAR(50),
    country VARCHAR(50),
    postal_code VARCHAR(50),
    loyalty_points INT
);
"""

products_schema = """
CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    category VARCHAR(50),
    brand TEXT,
    stock_quantity INT,
    color VARCHAR(50),
    size VARCHAR(20),
    weight DECIMAL(5,2),
    dimensions TEXT,
    release_date DATE,
    rating DECIMAL(3,1),
    reviews_count INT,
    seller_name TEXT,
    seller_rating DECIMAL(3,1),
    seller_reviews_count INT,
    shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
"""

orders_schema = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    total_price DECIMAL(10,2),
    order_date DATE,
    shipping_address VARCHAR(255),
    payment_method VARCHAR(20),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""

In [7]:
db_name = 'ecommerce.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [8]:
import sqlite3
import pandas as pd
import os



COLUMN_DATA_TYPES = {
    'customers': {
        'customer_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'phone_number': 'object',
        'address': 'object',
        'city': 'object',
        'country': 'object',
        'postal_code': 'object',
        'loyalty_points': 'int64'
    },
    'products': {
        'product_id': 'int64',
        'product_name': 'object',
        'description': 'object',
        'price': 'float64',
        'discount_percentage': 'float64',
        'category': 'object',
        'brand': 'object',
        'stock_quantity': 'int64',
        'color': 'object',
        'size': 'object',
        'weight': 'float64',
        'dimensions': 'object',
        'release_date': 'datetime64[ns]',
        'rating': 'float64',
        'reviews_count': 'int64',
        'seller_name': 'object',
        'seller_rating': 'float64',
        'seller_reviews_count': 'int64',
        'shipping_method': 'object',
        'shipping_cost': 'float64'
    },
    'orders': {
        'order_id': 'int64',
        'customer_id': 'int64',
        'product_id': 'int64',
        'quantity': 'int64',
        'unit_price': 'float64',
        'total_price': 'float64',
        'order_date': 'datetime64[ns]',
        'shipping_address': 'object',
        'payment_method': 'object',
        'status': 'object'
    }
}

# --- Database setup ---
db_name = 'ecommerce.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(customers_schema)
    cursor.execute(products_schema)
    cursor.execute(orders_schema)
    print("Tables 'customers', 'products', and 'orders' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/customers.csv': 'customers',
        '/content/products.csv': 'products',
        '/content/orders.csv': 'orders'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")



Database 'ecommerce.db' created and connected successfully. ✅
Tables 'customers', 'products', and 'orders' created successfully.

Processing '/content/customers.csv' for table 'customers'...
  -> Data from '/content/customers.csv' loaded into 'customers' table successfully.

Processing '/content/products.csv' for table 'products'...
  -> Data from '/content/products.csv' loaded into 'products' table successfully.

Processing '/content/orders.csv' for table 'orders'...
  -> Data from '/content/orders.csv' loaded into 'orders' table successfully.

Data committed to the database successfully. 🎉
Database connection closed.


# Tangent 2: You should setup your free API Key using Google's AI Studio

https://aistudio.google.com/

### Install Gen AI library

We will be installing of the google-generativeai package, the official Python SDK for the Gemini API.

In [9]:
!pip install google-genai

### Import required modules

In [10]:
from google import genai
from google.colab import userdata

In [12]:
genai_client = genai.Client(api_key=userdata.get('SECRETS_KEY'))

[link text](https://)# Tangent 3: You should learn about Prompt Engineering

## The Anatomy of an Effective Prompt: A Unified Framework

A fundamental advancement in prompt engineering is the realization that a prompt is not a monolithic question but a structured document composed of distinct components.

- **Role (or Persona):** This component defines who the model should be. Assigning a role, such as "You are a senior technical support specialist," constrains the model's vast knowledge base, forcing it to filter its response through a specific lens of expertise, tone, and style. This dramatically improves the coherence and domain-specificity of the output.


- **Context (or Background Information):** This provides the necessary background for the task. It can include user history, product documentation, previous conversation turns, or any other data that informs the query. Providing rich context is essential for generating relevant and personalized responses.


- **Task (or Instruction/Directive):** This is the core of the prompt—a clear, specific, and unambiguous statement of the action the model should perform. The use of direct action verbs (e.g., "Analyze," "Summarize," "Generate," "Classify") is critical for clarity.


- **Examples (or Shots):** These are high-quality examples of the desired input-output pattern. They are the foundation of few-shot prompting and are one of the most powerful tools for controlling output format and style. By showing the model exactly what is expected, examples enable a form of in-context learning.


- **Constraints (or Rules/Warnings):** This component defines the boundaries for the response. It specifies what the model should not do, such as avoiding certain topics, adhering to a word count, or refraining from using technical jargon. These "guardrails" are crucial for safety and brand alignment.


- **Output Format (or Structure):** This explicitly defines the structure of the desired output, such as JSON, Markdown, or a bulleted list. Specifying the format is vital for applications that need to programmatically parse the model's response, as it ensures the output is machine-readable and consistent

:To help the model distinguish between these different components, it is a best practice to use clear delimiters. Structuring the prompt with markers like Markdown headers (e.g., `###Instruction###`).

\[Unified Framework For An Effective Prompt](https://www.geeksforgeeks.org/data-science/a-unified-framework-for-an-effective-prompt/)

In [28]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's e-commerce database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`customers` table:**

```sql
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    address VARCHAR(50),
    city VARCHAR(50),
    country VARCHAR(50),
    postal_code VARCHAR(50),
    loyalty_points INT
);
```

**`products` table:**

```sql
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    category VARCHAR(50),
    brand TEXT,
    stock_quantity INT,
    color VARCHAR(50),
    size VARCHAR(20),
    weight DECIMAL(5,2),
    dimensions TEXT,
    release_date DATE,
    rating DECIMAL(3,1),
    reviews_count INT,
    seller_name TEXT,
    seller_rating DECIMAL(3,1),
    seller_reviews_count INT,
    shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
```

**`orders` table:**

```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    total_price DECIMAL(10,2),
    order_date DATE,
    shipping_address VARCHAR(255),
    payment_method VARCHAR(20),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all customers who live in Noida"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT * FROM customers WHERE city = 'Noida';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show me recent orders"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'recent' means? For example, 'in the last 7 days', 'this month', or 'since August 2025'."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Which warehouse has the most stock?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated."""

In [25]:
import json
def get_sql_query(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""

{prompt}
Here's the user query in english you need to work on:
{user_query}
"""
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output

In [15]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='ecommerce.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

In [16]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [29]:
text2sql(genai_client, prompt, "Show me the order count by country")

Input Token Count: 1530
Thoughts Token Count: 102
Output Token Count: 73
Total Token Count: 1705

Executing query on 'ecommerce.db':
SELECT T2.country, COUNT(T1.order_id) AS order_count FROM orders AS T1 INNER JOIN customers AS T2 ON T1.customer_id = T2.customer_id GROUP BY T2.country;
Query executed successfully.


,country,order_count
0,Afghanistan,9
1,Albania,8
2,American Samoa,3
3,Angola,5
4,Anguilla,2
...,...,...
122,United States,54
123,Uzbekistan,4
124,Venezuela,12
125,Vietnam,45


In [30]:
text2sql(genai_client, prompt, "What are my most popular products")

Input Token Count: 1529
Thoughts Token Count: 1985
Output Token Count: 73
Total Token Count: 3587

Executing query on 'ecommerce.db':
SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.quantity) DESC;
Query executed successfully.


,product_name
0,erat quisque erat eros viverra eget congue ege...
1,et ultrices posuere cubilia curae duis faucibu...
2,diam neque vestibulum eget vulputate ut ultric...
3,libero ut massa volutpat convallis morbi odio ...
4,in felis donec semper sapien a libero nam dui ...
...,...
929,dolor sit amet consectetuer adipiscing elit pr...
930,purus sit amet nulla quisque arcu libero rutru...
931,eros elementum pellentesque quisque porta volu...
932,viverra diam vitae quam suspendisse potenti nu...


In [31]:
text2sql(genai_client, prompt, "which country ranks in middle by total sales??")

Input Token Count: 1532
Thoughts Token Count: 2054
Output Token Count: 179
Total Token Count: 3765

Executing query on 'ecommerce.db':
WITH CountrySales AS (
    SELECT
        c.country,
        SUM(o.total_price) AS total_revenue
    FROM orders AS o
    JOIN customers AS c ON o.customer_id = c.customer_id
    GROUP BY c.country
),
RankedCountrySales AS (
    SELECT
        country,
        total_revenue,
        ROW_NUMBER() OVER (ORDER BY total_revenue ASC) AS rn,
        COUNT(*) OVER () AS total_countries_count
    FROM CountrySales
)
SELECT
    country
FROM RankedCountrySales
WHERE rn = (total_countries_count + 1) / 2;
Query executed successfully.


,country
0,Belarus


In [32]:
text2sql(genai_client, prompt, "rank and count of sales of India by total sales??")

Input Token Count: 1534
Thoughts Token Count: 1537
Output Token Count: 126
Total Token Count: 3197

Executing query on 'ecommerce.db':
WITH CountrySales AS (SELECT T2.country, SUM(T1.total_price) AS total_sales, COUNT(T1.order_id) AS sales_count FROM orders AS T1 JOIN customers AS T2 ON T1.customer_id = T2.customer_id GROUP BY T2.country) SELECT country, total_sales, sales_count, RANK() OVER (ORDER BY total_sales DESC) AS sales_rank FROM CountrySales WHERE country = 'India';
Query executed successfully.


""


In [33]:
text2sql(genai_client, prompt, "Give me the order count by day of month")

Input Token Count: 1532
Thoughts Token Count: 83
Output Token Count: 67
Total Token Count: 1682

Executing query on 'ecommerce.db':
SELECT strftime('%d', order_date) AS day_of_month, COUNT(order_id) AS order_count FROM orders GROUP BY day_of_month ORDER BY day_of_month ASC;
Query executed successfully.


,day_of_month,order_count
0,01,97
1,02,119
2,03,104
3,04,93
4,05,93
5,06,107
6,07,100
7,08,84
8,09,94
9,10,88


In [34]:
text2sql(genai_client, prompt, "On which day of the week do I get the most orders? Give me a detailed report.")

Input Token Count: 1542
Thoughts Token Count: 900
Output Token Count: 124
Total Token Count: 2566

Executing query on 'ecommerce.db':
SELECT CASE STRFTIME('%w', order_date) WHEN '0' THEN 'Sunday' WHEN '1' THEN 'Monday' WHEN '2' THEN 'Tuesday' WHEN '3' THEN 'Wednesday' WHEN '4' THEN 'Thursday' WHEN '5' THEN 'Friday' WHEN '6' THEN 'Saturday' END AS day_of_week, COUNT(order_id) AS total_orders FROM orders GROUP BY day_of_week ORDER BY total_orders DESC;
Query executed successfully.


,day_of_week,total_orders
0,Thursday,451
1,Sunday,444
2,Monday,435
3,Friday,435
4,Tuesday,431
5,Saturday,426
6,Wednesday,378


# What Next?

### **Retrieval-Augmented Generation (RAG)**

RAG is a powerful technique that combines the knowledge of a large language model with external data. This is especially useful when you need the model to answer questions about information it wasn't trained on.

* **Why it's a great next step:**
    * **Reduces Hallucinations:** The model's answers are grounded in the information you provide, making them more factual.
    * **Uses Real-Time Information:** You can constantly update your knowledge base with new information without having to retrain the model.
    * **Provides Citations:** You can show users the sources of the information used to generate the answer.



---


### **Fine-Tuning a Pre-Trained Model**

If you have a specific task and enough data, fine-tuning a smaller, open-source language model can be a powerful next step.

* **Why it's a good next step:**
    * **Improved Performance:** A fine-tuned model can achieve higher accuracy and more consistent outputs for your specific use case than a general-purpose model with just prompt engineering.
    * **Reduced Prompt Complexity:** You may be able to use much simpler prompts with a fine-tuned model.
    * **Potentially Lower Costs:** Using a smaller, fine-tuned model that you host yourself can sometimes be cheaper in the long run than making many API calls to a larger model.



---



### **Building More Complex Systems**

You can also think about building more sophisticated applications on top of the language model.

* **Agent-Based Systems:** Create "agents" that can use tools to perform actions. For example, an agent could be given access to a calculator, a search engine, or your company's internal APIs to break down complete tasks.
* **Multi-Step Reasoning:** For complex problems, you can break them down into smaller steps and have the language model solve each step in sequence. The output of one step can be used as the input for the next.

# Submission Details

Can you create dataset for employees, and share the results in a notebook?

In [35]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.9 MB/s eta 0:00:00


In [36]:
import pandas as pd
import sqlite3
import os
import random
from faker import Faker

fake = Faker()

num_employees = 500

employee_data = {
    "employee_id": list(range(1, num_employees + 1)),
    "first_name": [fake.first_name() for _ in range(num_employees)],
    "last_name": [fake.last_name() for _ in range(num_employees)],
    "email": [fake.email() for _ in range(num_employees)],
    "phone_number": [fake.phone_number() for _ in range(num_employees)],
    "department": [random.choice(["HR", "Finance", "Engineering", "Sales", "Marketing"]) for _ in range(num_employees)],
    "position": [random.choice(["Manager", "Analyst", "Engineer", "Intern", "Director"]) for _ in range(num_employees)],
    "salary": [round(random.uniform(30000, 150000), 2) for _ in range(num_employees)],
    "hire_date": [fake.date_between(start_date='-10y', end_date='today') for _ in range(num_employees)],
    "city": [fake.city() for _ in range(num_employees)],
    "country": [fake.country() for _ in range(num_employees)]
}

employees_df = pd.DataFrame(employee_data)

# Save to CSV
os.makedirs("data", exist_ok=True)
employees_csv_path = "data/employees.csv"
employees_df.to_csv(employees_csv_path, index=False)
print(f"Employee dataset saved to '{employees_csv_path}'")

# Define SQLite schema
employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    phone_number VARCHAR(50),
    department VARCHAR(50),
    position VARCHAR(50),
    salary DECIMAL(10,2),
    hire_date DATE,
    city VARCHAR(50),
    country VARCHAR(50)
);
"""

# Create SQLite database and load the CSV
db_name = 'company.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Create table
cursor.execute(employees_schema)
print("Employees table created successfully.")

# Load CSV into SQLite
employees_df.to_sql('employees', conn, if_exists='append', index=False)
print("Employee data loaded into SQLite successfully.")

# Verify with a simple SELECT
query = "SELECT * FROM employees LIMIT 5;"
cursor.execute(query)
rows = cursor.fetchall()

print("\nSample employee records:")
for row in rows:
    print(row)

conn.close()

Employee dataset saved to 'data/employees.csv'
Employees table created successfully.
Employee data loaded into SQLite successfully.

Sample employee records:
(1, 'Julia', 'Long', 'clarklaura@example.com', '(295)272-0406x77034', 'Marketing', 'Analyst', 84449.39, '2017-08-24', 'Petermouth', 'Netherlands')
(2, 'John', 'Miller', 'julie67@example.org', '001-651-879-4454x2112', 'Engineering', 'Analyst', 52295.92, '2019-11-04', 'Joemouth', 'Falkland Islands (Malvinas)')
(3, 'Darren', 'Lopez', 'cheryl93@example.com', '+1-787-344-1823x5750', 'Marketing', 'Manager', 97374.56, '2018-08-22', 'New Andrea', 'Mauritania')
(4, 'Paul', 'Wheeler', 'wtucker@example.org', '7468582036', 'Finance', 'Manager', 147617.84, '2019-02-23', 'North Derekborough', 'Turks and Caicos Islands')
(5, 'Matthew', 'Williams', 'jasmine68@example.com', '001-568-754-8370x1854', 'Finance', 'Manager', 40470.52, '2022-02-02', 'Andersonville', 'Argentina')
